In [0]:
# Restart runtime using 'Runtime' -> 'Restart runtime...'
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
else:
  print('Found GPU at: {}'.format(device_name))

#Clone git repos
import os
if os.getcwd() != '/content':
  %cd /content
!git clone https://github.com/tensorflow/models
!git clone https://github.com/laszlotomi/colab_demo.git 

#isntall dependencies
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

#Create proto files
%cd /content/models/research/
!protoc object_detection/protos/*.proto --python_out=.

#set environment variables
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim'

#Build app
!python setup.py build
!python setup.py install

#Run the test
!python object_detection/builders/model_builder_test.py

%cd object_detection/

In [0]:
#Copy required files
%cp /content/colab_demo/xml_to_csv.py /content/models/research/object_detection/xml_to_csv.py
%cp /content/colab_demo/generate_tfrecord.py generate_tfrecord.py
!mkdir training
%cp /content/colab_demo/labelmap.pbtxt training/labelmap.pbtxt
%cp /content/colab_demo/configs/faster_rcnn_inception_v2_coco.config training/faster_rcnn_inception_v2_coco.config
%cp /content/colab_demo/configs/ssd_inception_v2_coco.config training/ssd_inception_v2_coco.config
%cp /content/models/research/object_detection/legacy/train.py train.py 
%cp -R /content/colab_demo/images /content/models/research/object_detection/

In [0]:
#Convert xml to csv
!python xml_to_csv.py
# for test data
!python generate_tfrecord.py --csv_input=data/test_labels.csv  --output_path=test.record  --image_dir=/content/colab_demo/images/test
# for train data
!python generate_tfrecord.py --csv_input=data/train_labels.csv  --output_path=train.record  --image_dir=/content/colab_demo/images/train


In [0]:
#cd /content/colab_demo/
#!git init

In [0]:
model="faster_rcnn_inception_v2_coco_2018_01_28.tar.gz"
#model="ssd_mobilenet_v1_coco_2018_01_28.tar.gz"
url= os.path.join("http://download.tensorflow.org/models/object_detection/", model)
!wget $url
!tar -xvf $model

In [0]:
LOG_DIR = 'training'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
# Install
! npm install -g localtunnel
! npm i -g npm
# Tunnel port 6006 (TensorBoard assumed running)
get_ipython().system_raw('lt --port 6006 >> url1.txt 2>&1 &')
# Get url
! cat url1.txt

In [0]:
config_file = "faster_rcnn_inception_v2_coco.config"
path = "training/" + config_file
!python train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/faster_rcnn_inception_v2_coco.config

In [0]:
# %rm -r /content/models/research/object_detection/training
# %mkdir /content/models/research/object_detection/training
# %cp /content/colab_demo/labelmap.pbtxt training/labelmap.pbtxt
# # %cp /content/colab_demo/configs/faster_rcnn_inception_v2_coco.config training/faster_rcnn_inception_v2_coco.config

In [0]:
!python export_inference_graph.py \
--input_type image_tensor \
--pipeline_config_path training/faster_rcnn_inception_v2_coco.config \
--trained_checkpoint_prefix training/model.ckpt-0 \
--output_directory My_exp_graph/

!zip -r MY_exp_g.zip My_exp_graph


In [0]:
MODEL_NAME = 'My_exp_graph'
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = 'training/labelmap.pbtxt'
NUM_CLASSES = 1 

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops
%matplotlib inline
from utils import label_map_util
from utils import visualization_utils as vis_util

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

# Loading label map
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

#Helper code
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

import glob
TEST_IMAGE_PATHS = []
for filename in glob.iglob('./test_images/*.JPEG', recursive=True):
    TEST_IMAGE_PATHS.append(filename)
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)



In [0]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [0]:
for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=4)
  #print(category_index)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)
  plt.grid(False)
  